In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('data/cleaned_df.csv')
df.head()

,order_id,customer_unique_id,product_id,seller_id,order_date,price,freight_value,customer_zip_code_prefix,customer_city,customer_state
0,e481f51cbdc54678b7cc49136f2d6af7,7c396fd4830fd04220f754e42b4e5bff,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-02 10:56:33,29.99,8.72,3149,sao paulo,SP
1,53cdb2fc8bc7dce0b6741e2150273451,af07308b275d755c9edb36a90c618231,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-24 20:41:37,118.70,22.76,47813,barreiras,BA
2,47770eb9100c2d0c44946d9cf07ec65d,3a653a41f6f9fc3d2a113cf8398680e8,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-08 08:38:49,159.90,19.22,75265,vianopolis,GO
3,949d5b44dbf5de918fe9c16f97b45f8a,7c142cf63193a1473d2e66489a9ae977,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,2017-11-18 19:28:06,45.00,27.20,59296,sao goncalo do amarante,RN
4,ad21c59c0840e6cb83a9ceb5573f8159,72632f0f9dd73dfee390c9b22eb56dd6,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,2018-02-13 21:18:39,19.90,8.72,9195,santo andre,SP


In [3]:
group_item_cust = df\
                    .groupby(['customer_unique_id', 'product_id'])['order_date']\
                    .count()\
                    .reset_index()\
                    .rename(columns={'order_date':'quantity'})
print(group_item_cust.info())
group_item_cust.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99505 entries, 0 to 99504
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_unique_id  99505 non-null  object
 1   product_id          99505 non-null  object
 2   quantity            99505 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 2.3+ MB
None


,customer_unique_id,product_id,quantity
0,0000366f3b9a7992bf8c76cfdf3221e2,372645c7439f9661fbbacfd129aa92ec,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,5099f7000472b634fea8304448d20825,1
2,0000f46a3911fa3c0805444483337064,64b488de448a5324c4134ea39c28a34b,1
3,0000f6ccb0745a6a4b88665a16c9f078,2345a354a6f2033609bbf62bf5be9ef6,1
4,0004aac84e0df4da2b147fca70cf8255,c72e18b3fe2739b8d24ebf3102450f37,1


In [ ]:
chunk_size = 1000
chunks = [group_item_cust.iloc[i:i+chunk_size] for i in range(0, len(group_item_cust), chunk_size)]

# Initialize an empty DataFrame to store the results
customer_item_matrix = pd.DataFrame()
count = 0
for chunk in chunks:
    pivot_chunk = chunk.pivot_table(
        index='customer_unique_id', 
        columns='product_id', 
        values='quantity', 
        aggfunc='sum', 
        fill_value=0
    )
    count = count + 1
    print(count)
    customer_item_matrix = pd.concat([customer_item_matrix, pivot_chunk], axis=0)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84


In [ ]:
customer_item_matrix.head()

In [ ]:
customer_item_matrix = customer_item_matrix.applymap(lambda x: 1 if x > 0 else 0)

customer_item_matrix.head()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

item_item_sim_matrix = pd.DataFrame(cosine_similarity(customer_item_matrix.T))

item_item_sim_matrix.columns = customer_item_matrix.T.index

item_item_sim_matrix['StockCode'] = customer_item_matrix.T.index
item_item_sim_matrix = item_item_sim_matrix.set_index('StockCode')

item_item_sim_matrix.head()

In [ ]:
item_item_sim_matrix.to_csv('./data/item-item_similarity_matrix.csv', index=False)